In [ ]:
pip install llama-index openai chromadb requests beautifulsoup4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-legacy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 10.4 MB/s eta 0:00:0

VECTOR DB CHROMA

In [ ]:
from chromadb import Client
from chromadb.config import Settings

# Initialize Chroma client
client = Client(Settings(
    persist_directory='./chroma_db',
))

# Create or get a collection
collection_name = 'alice_adventures'
if collection_name in client.list_collections():
    collection = client.get_collection(name=collection_name)
else:
    collection = client.create_collection(name=collection_name)


# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

UniqueConstraintError: Collection alice_adventures already exists

Download and Process the Document



In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the document
url = 'https://www.gutenberg.org/cache/epub/11/pg11.html'

# Download the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoupGroup 3 d
soup = BeautifulSoup(html_content, 'html.parser')
text = soup.get_text()

# Optional: Clean up the text (remove unwanted parts, etc.)
# For simplicity, we are using the raw text here


Index the Document



In [ ]:
from openai import OpenAI

client = OpenAI(api_key='sk-')

# Function to get embeddings
def get_embeddings(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return response.data[0].embedding

# Add document to Chroma
chunk_size = 1000  # Adjust chunk size based on your needs
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

for idx, chunk in enumerate(chunks):
    embedding = get_embeddings(chunk)
    collection.add(
        ids=[f'chunk_{idx}'],
        documents=[{'id': f'chunk_{idx}', 'text': chunk}],
        embeddings=[embedding]
    )


ERROR:chromadb.db.mixins.embeddings_queue:Exception occurred invoking consumer for subscription 8cdd1afd839c44989aba2cfcd160c3f6to topic persistent://default/default/557eb53d-aee6-4398-940a-8ed932bb234c Error binding parameter 1 - probably unsupported type.


CreateEmbeddingResponse(data=[Embedding(embedding=[0.011592376977205276, -0.01982906460762024, -0.019285257905721664, -0.041090600192546844, -0.022627683356404305, 0.035572946071624756, 0.0033109234645962715, -0.014563420787453651, -0.032867174595594406, -0.011002146638929844, 0.003181603504344821, 0.016155051067471504, -0.005610498134046793, -0.01237492822110653, 0.006698112469166517, 0.020518772304058075, 0.03339771553874016, -0.010418549180030823, -0.010683820582926273, -0.02398056909441948, -0.011366895399987698, 0.008574910461902618, -0.003909443039447069, -0.01555818971246481, 0.010537921451032162, 0.0075270868837833405, 0.016367269679903984, -0.021619649603962898, 0.007294973824173212, 0.0019199044909328222, 0.0007647288148291409, 0.01852923445403576, -0.030187929049134254, -0.017839526757597923, -0.017839526757597923, -0.018555760383605957, 0.0013578599318861961, -0.00793162640184164, 0.005169483833014965, -0.01631421409547329, 0.02278684638440609, 0.017202874645590782, 0.01460

Set Up LlamaIndex for Retrieval



In [ ]:
from llama_index import GPTIndex, SimpleDocument

# Create a GPTIndex object
index = GPTIndex()

# Add documents to the index
for idx, chunk in enumerate(chunks):
    index.add_document(SimpleDocument(
        id=f'chunk_{idx}',
        text=chunk
    ))


Create a Query Function



In [ ]:
from openai import OpenAI

# Function to perform retrieval and generation
def retrieve_and_generate(query):
    # Retrieve relevant documents using LlamaIndex
    retrieved_docs = index.query(query)

    # If no documents are retrieved, return a default message
    if not retrieved_docs:
        return "No relevant documents found."

    # Generate a response using OpenAI's GPT
    prompt = f"Based on the following documents, answer the question: {query}\n\nDocuments:\n" + "\n".join(doc['text'] for doc in retrieved_docs)
    response = OpenAI.Completion.create(
        model="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()


Test the Pipeline



In [ ]:
query = "What is the main theme of the book?"
response = retrieve_and_generate(query)
print(response)